In [1]:
import os,sys
from importlib import reload
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
#import torch
print(os.getpid())
sys.path.append(r'/lab/weissman_imaging/puzheng/Softwares/') # parent folder of ChromAn

1623088


In [2]:
pool_folder = r'/lab/solexa_weissman/puzheng/MERFISH_Probes/PE_TS/NewTargetSites'
print(pool_folder)
library_folder = os.path.join(pool_folder, 'IntBC_MERFISH_v2')
print(library_folder)
if not os.path.exists(library_folder):
    os.makedirs(library_folder)

/lab/solexa_weissman/puzheng/MERFISH_Probes/PE_TS/NewTargetSites
/lab/solexa_weissman/puzheng/MERFISH_Probes/PE_TS/NewTargetSites/IntBC_MERFISH_v2


In [3]:
# check integration barcode designs:
target_site_filename = os.path.join(library_folder, 'PETS_orthoSeq_targets.xlsx')
# reload target sites:
target_df = pd.read_excel(target_site_filename)
# determine number
num_unique_probes = np.sum(['_rc' not in _n for _n in target_df['name']])
print(f"Number of unique target_sites: {num_unique_probes}")

Number of unique target_sites: 2171


## import codebook

In [4]:
sel_barcode_filename = os.path.join(library_folder, 'sel_merfish_barcodes.npy')
sel_barcodes = np.load(sel_barcode_filename)
print(len(sel_barcodes))

2171


## import readouts

In [5]:
readout_folder = r'/lab/solexa_weissman/puzheng/References/Readouts'
readout_filename = os.path.join(readout_folder, 'perturbfish_bits_4base_extended.csv')
readout_df = pd.read_csv(readout_filename)
readout_df

,Name,Seq tail,Description,Reverse Complement,short_name,Seq_15mer_Tm45,Seq_15mer_Tm45_RC
0,R1__NDB_784,TAGAAATATCCGAGTGGCAT,MERFISH,ATGCCACTCGGATATTTCTA,R1,AATATCCGAGTGGCA,TGCCACTCGGATATT
1,R2__NDB_755,TGCGACATGTTATGCTAGGC,MERFISH,GCCTAGCATAACATGTCGCA,R2,TGCGACATGTTATGC,GCATAACATGTCGCA
2,R3__NDB_759,TGACTGTCGTTGACTCTCGG,MERFISH,CCGAGAGTCAACGACAGTCA,R3,TGTCGTTGACTCTCG,CGAGAGTCAACGACA
3,R4__NDB_826,ATATGTCGATGTCCTTAGAC,MERFISH,GTCTAAGGACATCGACATAT,R4,TGTCGATGTCCTTAG,CTAAGGACATCGACA
4,R5__NDB_578,AGGCGATACATGAGAGGCGT,MERFISH,ACGCCTCTCATGTATCGCCT,R5,AGGCGATACATGAGA,TCTCATGTATCGCCT
...,...,...,...,...,...,...,...
189,R190__NDB_941,GCAATAGCGGCACCTTTCAC,Extra,GTGAAAGGTGCCGCTATTGC,R190,AATAGCGGCACCTTT,AAAGGTGCCGCTATT
190,R191__NDB_846,GGCTATTAGTCGGTCTGATA,Extra,TATCAGACCGACTAATAGCC,R191,GGCTATTAGTCGGTC,GACCGACTAATAGCC
191,R192__NDB_775,GTTACTCGACCCGCGTGCAA,Extra,TTGCACGCGGGTCGAGTAAC,R192,GTTACTCGACCCGCG,CGCGGGTCGAGTAAC
192,R193__NDB_439,TCTCGTAGAGATCCTCACCG,Extra,CGGTGAGGATCTCTACGAGA,R193,CGTAGAGATCCTCAC,GTGAGGATCTCTACG


In [6]:
# select readouts
sel_readouts = pd.DataFrame({'Name':readout_df['Name'].values[0:24], 'Sequence':readout_df['Seq tail'][0:24].values})
sel_readouts # these are the sequences in the probes

,Name,Sequence
0,R1__NDB_784,TAGAAATATCCGAGTGGCAT
1,R2__NDB_755,TGCGACATGTTATGCTAGGC
2,R3__NDB_759,TGACTGTCGTTGACTCTCGG
3,R4__NDB_826,ATATGTCGATGTCCTTAGAC
4,R5__NDB_578,AGGCGATACATGAGAGGCGT
5,R6__NDB_795,GGGCGTCTGTTACACGTTTG
6,R7__NDB_376,GACTTTGTGGGTACGTTGAT
7,R8__NDB_713,GGTGGTTGTCACTTTCCGAA
8,R9__NDB_777,CGGCGAATTTGCTTTCGTTA
9,R10__NDB_865,GATCCCGGTCGGATATGTGA


## import primers

In [7]:
from ChromAn.src.library_tools.primers import load_fasta_to_DataFrame
import ChromAn
from Bio.SeqIO.FastaIO import SimpleFastaParser

In [8]:
reload(ChromAn.src.library_tools.primers)

<module 'ChromAn.src.library_tools.primers' from '/lab/weissman_imaging/puzheng/Softwares/ChromAn/src/library_tools/primers.py'>

In [9]:
primer_folder = r'/lab/solexa_weissman/puzheng/References/Primers'
# fwd
forward_primers = load_fasta_to_DataFrame(os.path.join(primer_folder, 'forward_primers_keep.fasta'))
reverse_primers = load_fasta_to_DataFrame(os.path.join(primer_folder, 'reverse_primers_keep.fasta'))

In [10]:
forward_primers

,Name,Sequence
0,W1A01_primer_0,CGGCTCGCAGCGTGTAAACG
1,W1A03_primer_2,CCCGCAATGGCTGACAACCG
2,W1A05_primer_4,CATTCAGCATTGCGCAACGG
3,W1A07_primer_6,CGCAAACTGGTGCGGAAGGC
4,W1A09_primer_8,TTGTTGAGGCGGCGGAAGTC
5,W1A11_primer_10,TTGCTAGCGTGGGCCAATCG
6,W1B01_primer_12,TTGGGCGTGCACGTCAATTC
7,W1B03_primer_14,GACATCCAGGCGCGGACAAC
8,W1B05_primer_16,CGGGTTTCGTTGCGCACACC
9,W1B07_primer_18,CACGTGGCCTCTCGCACATC


## design strategy 1
> 30nt design (6+1 non-overlapping probes)
>> 135 246 145 236 124 356 + 123,456 compete

In [11]:
probeLoc_30ntNO_2_bits = {
    0:[135], #start from 0, instead of 33
    30:[246],
    60:[145],
    90:[236],
    120:[124],
    150:[356],
    #237:[123,456], Modify: 30nt part don't design merfish
}

In [12]:
import re
from Bio.Seq import reverse_complement

# internalize
probeLoc_2_bits = probeLoc_30ntNO_2_bits

probe_len = 30
primer_len = 20

readout_junction = 't'
name_junction = '_'

# primers
fwd_primer = forward_primers.loc[forward_primers['Name']=='W1A03_primer_2']
rev_primer = reverse_primers.loc[reverse_primers['Name']=='W1A10_primer_9']

probe_seqs = []
probe_names = []

for _i, _row in tqdm(target_df.iterrows()):
    if _i >= len(sel_barcodes):
        break
    #print(_i)
    _intBC_name = _row['name']
    _intBC_seq = _row['seq']
    #_intBC_seq = ''.join(re.split('[atcg]+', _intBC_seq, )[1:-1])
    _intBC_barcode = sel_barcodes[_i]
    _intBC_readouts = sel_readouts.iloc[_intBC_barcode]
    for _ip, (_loc, _bits_list) in enumerate(probeLoc_2_bits.items()):
        
        _targeting_region = reverse_complement(_intBC_seq[_loc:_loc+probe_len])
        _target_name = f"{_intBC_name}_probe_{_ip}_loc_{_loc}"
        #print(_loc, _targeting_region)
        for _bits in _bits_list:
            _bits = np.array([*str(_bits)], dtype=np.int32) - 1
            _probe_readout_seq = readout_junction.join(list(_intBC_readouts['Sequence'].values[_bits]))
            _probe_readout_name = 'readout' + name_junction + ','.join(list(_intBC_readouts['Name'].values[_bits]))
            # append probe info
            probe_seqs.append(fwd_primer['Sequence'].values[0][-primer_len:] + readout_junction \
                              + _targeting_region + readout_junction + _probe_readout_seq\
                              + readout_junction + reverse_complement(rev_primer['Sequence'].values[0][-primer_len:]) )
            probe_names.append(fwd_primer['Name'].values[0] + name_junction \
                               + _target_name + name_junction \
                               + _probe_readout_name + name_junction + rev_primer['Name'].values[0])
probe_30ntNO = pd.DataFrame({'Name':probe_names, 'Sequence': probe_seqs})
probe_30ntNO  

0it [00:00, ?it/s]

,Name,Sequence
0,W1A03_primer_2_intID1_probe_0_loc_0_readout_R1...,CCCGCAATGGCTGACAACCGtTTACGTCGTGGTTGGGCCCTCACAA...
1,W1A03_primer_2_intID1_probe_1_loc_30_readout_R...,CCCGCAATGGCTGACAACCGtATTTACGACTGAACGCAGCAACAAT...
2,W1A03_primer_2_intID1_probe_2_loc_60_readout_R...,CCCGCAATGGCTGACAACCGtGTGCGCCACAGATATAGGGCGAATC...
3,W1A03_primer_2_intID1_probe_3_loc_90_readout_R...,CCCGCAATGGCTGACAACCGtTAATACGGAGGAATTACAGCTTCAA...
4,W1A03_primer_2_intID1_probe_4_loc_120_readout_...,CCCGCAATGGCTGACAACCGtAGACAGGGCGGCTACCTCAAGTAAC...
...,...,...
13021,W1A03_primer_2_intID2171_probe_1_loc_30_readou...,CCCGCAATGGCTGACAACCGtGCTCAGGACTCCGATGAGATGACCC...
13022,W1A03_primer_2_intID2171_probe_2_loc_60_readou...,CCCGCAATGGCTGACAACCGtGAGACTCCGCGTTGCACATCCAACG...
13023,W1A03_primer_2_intID2171_probe_3_loc_90_readou...,CCCGCAATGGCTGACAACCGtGAAGAGCATTAAAATGAGCATCGAC...
13024,W1A03_primer_2_intID2171_probe_4_loc_120_reado...,CCCGCAATGGCTGACAACCGtCGACGGAGAGTCTGAGGGTTAAGCA...


In [15]:
probe_30ntNO['Sequence'][0].split('t')

['CCCGCAATGGCTGACAACCG',
 'TTACGTCGTGGTTGGGCCCTCACAAATACA',
 'TAGAAATATCCGAGTGGCAT',
 'TGACTGTCGTTGACTCTCGG',
 'AGGCGATACATGAGAGGCGT',
 'CGGAAACCATGCGGCAATCC']

In [19]:
probe_30ntNO['Sequence'][1000][21:51]

'CAAGTAGGTGCGAAAATTATCCACGTGCTC'

In [22]:
2171 * 6, len(probe_30ntNO['Sequence'][0])

(13026, 135)

In [21]:
# save probes
import time
save_probe_30ntNO_filename = os.path.join(library_folder,
    f"{time.localtime().tm_year}_{time.localtime().tm_mon}_{time.localtime().tm_mday}_probe_30ntIntBC.xlsx")
print(save_probe_30ntNO_filename)
probe_30ntNO.to_excel(save_probe_30ntNO_filename, index=None)

/lab/solexa_weissman/puzheng/MERFISH_Probes/PE_TS/NewTargetSites/IntBC_MERFISH_v2/2024_2_7_probe_30ntIntBC.xlsx
